# Chapter 7 — Multi-Agent: AgentTool

Wrap an agent as a **tool** for another agent using `AgentTool`.

This allows a parent agent to call a sub-agent like a function — on demand, with specific inputs.

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()
assert os.environ.get("GOOGLE_API_KEY"), "Set GOOGLE_API_KEY first"
print("Google API Key set:", bool(os.environ.get("GOOGLE_API_KEY")))

## Create a Specialist Agent and Wrap It as a Tool

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import AgentTool

# A specialist agent for translating text
translator_agent = Agent(
    name="translator",
    model="gemini-2.0-flash",
    description="Translates text between languages.",
    instruction="You are a professional translator. Translate the given text to the requested language. Preserve tone and meaning.",
)

# A specialist agent for summarizing
summarizer_agent = Agent(
    name="summarizer",
    model="gemini-2.0-flash",
    description="Summarizes long text into concise summaries.",
    instruction="You are a summarization expert. Create clear, concise summaries that capture the key points of the given text.",
)

# Wrap them as tools
translator_tool = AgentTool(agent=translator_agent)
summarizer_tool = AgentTool(agent=summarizer_agent)

print("Agent tools created: translator, summarizer")

## Create the Parent Agent

In [ ]:
# Parent agent that uses sub-agents as tools
assistant = Agent(
    name="assistant",
    model="gemini-2.0-flash",
    description="A versatile assistant that can translate and summarize.",
    instruction="""You are a helpful assistant with access to translation and summarization tools.
Use the translator tool when the user needs text translated.
Use the summarizer tool when the user wants a summary.
You can chain these tools — e.g., summarize first, then translate the summary.""",
    tools=[translator_tool, summarizer_tool],
)

print("Parent assistant ready with AgentTools:", [t.agent.name for t in assistant.tools])

In [ ]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
runner = Runner(agent=assistant, app_name="agent_tool_demo", session_service=session_service)

session = await session_service.create_session(app_name="agent_tool_demo", user_id="user1")

# Test: translation
request = types.Content(
    role="user",
    parts=[types.Part(text="Translate 'The future of AI is collaborative intelligence' to Spanish and French.")]
)

async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=request):
    if event.content and event.content.parts:
        print(f"[{event.author}]: {event.content.parts[0].text}")

In [ ]:
# Test: chaining — summarize then translate
request2 = types.Content(
    role="user",
    parts=[types.Part(text="""Summarize the following text, then translate the summary to Japanese:

Artificial intelligence has transformed numerous industries over the past decade. From healthcare
diagnostics to autonomous vehicles, AI systems are becoming increasingly sophisticated. Machine
learning models can now process vast amounts of data to identify patterns that humans might miss.
However, the rapid advancement also raises important ethical questions about privacy, bias, and
the future of work. Researchers and policymakers are working together to develop frameworks
that ensure AI benefits society while minimizing potential harms.""")]
)

async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=request2):
    if event.content and event.content.parts:
        print(f"[{event.author}]: {event.content.parts[0].text}")

## Key Takeaways

- **`AgentTool`**: Wraps an agent so it can be called like a tool by another agent
- The parent agent decides **when** and **how** to invoke each agent-tool
- Enables composable, modular agent architectures
- Tools can be chained: the parent orchestrates multi-step workflows